# Indo coral cover calculator
Reads csv from CoraNet and prints coral cover for each site

TODO:
- Check all sites found
- Fix the labels which can't be found in final print out

In [4]:
import pandas as pd
import os

In [6]:
#@title Set Paths
base_dir = os.getenv('BASE_DIR')
if not base_dir:
    raise ValueError("BASE_DIR environment variable is not set.")

In [29]:
# Load the CSV
coral_cover_csv = os.path.join(base_dir, "marrs_acoustics/data/additionals", "coral_cover.csv")
df = pd.read_csv(coral_cover_csv)  # Adjust delimiter as needed

/tmp/ipykernel_127571/3190429073.py:3: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(coral_cover_csv)  # Adjust delimiter as needed


In [30]:
def title_case_and_remove_spaces(x: str) -> str:
    # Split by whitespace, capitalise first letter of each word, then join without spaces
    words = x.strip().split()
    words = [w.capitalize() for w in words]
    return ''.join(words)  # join without spaces

df['site'] = df.apply(
    lambda row: f"{title_case_and_remove_spaces(row['Project'])}_{title_case_and_remove_spaces(row['Site (broad)'])}",
    axis=1
)

# Insert 'site' column to the right of 'Site (broad)'
site_broad_index = df.columns.get_loc('Site (broad)')
df.insert(site_broad_index+1, 'site', df.pop('site'))


In [32]:
# CoralNet hard coral lookup table for Label entries,
hard_coral_dict = {
    'ALVEn':'Hard coral', 'ANACn':'Hard coral', 'ASTRn':'Hard coral', 'bASTRn':'Hard coral', 'bCBAn':'Hard coral', 
    'bCBArs':'Hard coral', 'bCDAn':'Hard coral', 'bCDArs':'Hard coral', 'bCEn':'Hard coral', 'bCFn':'Hard coral',
    'bCMn':'Hard coral', 'bCTAn':'Hard coral', 'bCTArs':'Hard coral', 'bCYPHn':'Hard coral', 'bDIPLn':'Hard coral',
    'bECHIn':'Hard coral', 'bFUNGn':'Hard coral', 'bFVIAn':'Hard coral', 'bFVTSn':'Hard coral', 'bGALXn':'Hard coral',
    'bGONPn':'Hard coral', 'bGONSn':'Hard coral', 'bHERPn':'Hard coral', 'bHYDNn':'Hard coral', 'bISOPn':'Hard coral',
    'bLOBOn':'Hard coral', 'bLPTAn':'Hard coral', 'bLPTOn':'Hard coral', 'bMERUn':'Hard coral', 'bMONPn':'Hard coral',
    'bMYCEn':'Hard coral', 'bOXYPn':'Hard coral', 'bPACHn':'Hard coral', 'bPAVOn':'Hard coral', 'bPECTn':'Hard coral',
    'bPLATn':'Hard coral', 'bPOCIn':'Hard coral', 'bPORIn':'Hard coral', 'bPSAMn':'Hard coral', 'bSERIn':'Hard coral',
    'bStyloph':'Hard coral', 'CAULn':'Hard coral', 'CBAn':'Hard coral', 'CBArs':'Hard coral', 'CBNAn':'Hard coral',
    'CBNArs':'Hard coral', 'CDAn':'Hard coral', 'CDArs':'Hard coral', 'CDNAn':'Hard coral', 'CEn':'Hard coral',
    'CErs':'Hard coral', 'CFn':'Hard coral', 'CFrs':'Hard coral', 'CLn':'Hard coral', 'CMn':'Hard coral',
    'COELn':'Hard coral', 'COSCn':'Hard coral', 'CSn':'Hard coral', 'CSrs':'Hard coral', 'CTAn':'Hard coral',
    'CTArs':'Hard coral', 'CTENn':'Hard coral', 'CYPHn':'Hard coral', 'dCBAn':'Hard coral', 'dCBArs':'Hard coral',
    'dCBNAn':'Hard coral', 'dCBNArs':'Hard coral', 'dCDAn':'Hard coral', 'dCDArs':'Hard coral', 'dCEn':'Hard coral',
    'dCFn':'Hard coral', 'dCMn':'Hard coral', 'dCSn':'Hard coral', 'dCTAn':'Hard coral', 'dCTArs':'Hard coral',
    'DIPLn':'Hard coral', 'ECHIn':'Hard coral', 'EUPHn':'Hard coral', 'FUNGn':'Hard coral', 'FVIAn':'Hard coral',
    'FVTSn':'Hard coral', 'GALXn':'Hard coral', 'GONPn':'Hard coral', 'GONSn':'Hard coral', 'HALOn':'Hard coral',
    'HERPn':'Hard coral', 'HYDNn':'Hard coral', 'ISOPn':'Hard coral', 'LITHn':'Hard coral', 'LOBOn':'Hard coral',
    'LPTAn':'Hard coral', 'LPTOn':'Hard coral', 'MERUn':'Hard coral', 'MONPn':'Hard coral', 'MONSn':'Hard coral',
    'MYCEn':'Hard coral', 'OULOn':'Hard coral', 'OXYPn':'Hard coral', 'PACHn':'Hard coral', 'PAVOn':'Hard coral',
    'PECTn':'Hard coral', 'PLATn':'Hard coral', 'POCIn':'Hard coral', 'POCIrs':'Hard coral', 'POLYn':'Hard coral',
    'PORIn':'Hard coral', 'PORIrs':'Hard coral', 'PSAMn':'Hard coral', 'SERIn':'Hard coral', 'SERIrs':'Hard coral',
    'SNDLn':'Hard coral', 'STYLn':'Hard coral', 'STYLrs':'Hard coral', 'SYMPn':'Hard coral'
}

### Check all sites are present and a nearby survey date

In [33]:
# Convert the Date column to datetime if not already
df['Date'] = pd.to_datetime(df['Date'])

# Define the target date
target_date = pd.to_datetime('2022-09-01')

# For each site, find the date closest to the target date
closest_dates = df.groupby('site')['Date'].apply(
    lambda dates: dates.iloc[(dates - target_date).abs().argmin()]
)

# Convert this Series into a dict for easy lookup
closest_dates_dict = closest_dates.to_dict()

# Filter the dataframe to keep only rows with the chosen closest date for their site
filtered_df = df[df.apply(lambda row: row['Date'] == closest_dates_dict[row['site']], axis=1)]

#Print each unique 'Site (broad)' and associated unique 'Date' entries in filtered_df
sites = filtered_df['site'].unique()
for site in sites:
    print(f"Site: {site}")
    dates = filtered_df.loc[df['site'] == site, 'Date'].unique()
    for d in dates:
        print(f"  Date: {d}")


Site: Bontosua_BadControl1
  Date: 2022-11-10 00:00:00
Site: Bontosua_BadControl2
  Date: 2022-11-07 00:00:00
Site: Bontosua_BadControl3
  Date: 2022-11-12 00:00:00
Site: Bontosua_Block10
  Date: 2022-11-09 00:00:00
Site: Bontosua_Block11
  Date: 2022-11-10 00:00:00
Site: Bontosua_Block1
  Date: 2022-11-09 00:00:00
Site: Bontosua_Block2
  Date: 2022-11-08 00:00:00
Site: Bontosua_Block3
  Date: 2022-11-02 00:00:00
Site: Bontosua_Block4
  Date: 2022-11-14 00:00:00
Site: Bontosua_Block5
  Date: 2022-11-02 00:00:00
Site: Bontosua_Block6
  Date: 2022-11-03 00:00:00
Site: Bontosua_Block7
  Date: 2022-11-03 00:00:00
Site: Bontosua_Block8
  Date: 2022-11-17 00:00:00
Site: Bontosua_Block9
  Date: 2022-11-15 00:00:00
Site: Bontosua_CentralEastSector
  Date: 2022-10-25 00:00:00
Site: Bontosua_CentralSector
  Date: 2022-10-25 00:00:00
Site: Bontosua_GoodControl1
  Date: 2022-11-12 00:00:00
Site: Bontosua_GoodControl2
  Date: 2022-11-08 00:00:00
Site: Bontosua_GoodControl3
  Date: 2022-11-16 00:00:

### Print total entries for each site and proportion that are hard coral

In [35]:
# Keep track of labels not found in hard_coral_dict to print later
unknown_labels = set()

for site in sites:
    site_df = filtered_df[filtered_df['site'] == site]
    total_rows = len(site_df)
    
    # Map each label to whether it's hard coral or not
    def is_hard_coral(label: str) -> bool:
        if label in hard_coral_dict:
            return True
        else:
            # Record unknown labels
            if pd.notna(label) and label not in hard_coral_dict and label not in unknown_labels:
                unknown_labels.add(label)
            return False
    
    hard_coral_count = sum(site_df['Label'].apply(is_hard_coral))
    
    proportion = hard_coral_count / total_rows if total_rows > 0 else 0
    print(f"Site: {site}, Total Rows: {total_rows}, Proportion Hard Coral: {proportion:.2f}")

# Print unknown labels
if unknown_labels:
    print("\nLabels not found in hard coral dictionary:")
    for lbl in unknown_labels:
        print(lbl)


Site: Bontosua_BadControl1, Total Rows: 2020, Proportion Hard Coral: 0.80
Site: Bontosua_BadControl2, Total Rows: 2040, Proportion Hard Coral: 0.60
Site: Bontosua_BadControl3, Total Rows: 2040, Proportion Hard Coral: 0.23
Site: Bontosua_Block10, Total Rows: 2040, Proportion Hard Coral: 0.84
Site: Bontosua_Block11, Total Rows: 2040, Proportion Hard Coral: 0.60
Site: Bontosua_Block1, Total Rows: 2040, Proportion Hard Coral: 0.72
Site: Bontosua_Block2, Total Rows: 2040, Proportion Hard Coral: 0.80
Site: Bontosua_Block3, Total Rows: 2040, Proportion Hard Coral: 0.78
Site: Bontosua_Block4, Total Rows: 2040, Proportion Hard Coral: 0.83
Site: Bontosua_Block5, Total Rows: 2040, Proportion Hard Coral: 0.68
Site: Bontosua_Block6, Total Rows: 2040, Proportion Hard Coral: 0.53
Site: Bontosua_Block7, Total Rows: 2040, Proportion Hard Coral: 0.72
Site: Bontosua_Block8, Total Rows: 2040, Proportion Hard Coral: 0.49
Site: Bontosua_Block9, Total Rows: 2040, Proportion Hard Coral: 0.52
Site: Bontosua_Ce